In [ ]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [ ]:
submission_files = [
    "/opt/ml/level2_objectdetection-cv-01/work_dirs/swin-t-p4-w7_cascade_rcnn_fpn_adamW_focalLoss_CosRes_mstrain/submission_latest.csv",
    "/opt/ml/level2_objectdetection-cv-01/work_dirs/swin-t-p4-w7_cascade_rcnn_Non_Local_fpn_adamW_focalLoss_CosRes_mstrain/submission_latest.csv"
    ]
submission_df = [pd.read_csv(file) for file in submission_files]

In [ ]:
submission_df

In [ ]:
annotation = '../dataset/test.json'
coco = COCO(annotation)
image_ids = submission_df[0]['image_id'].tolist()

In [ ]:
prediction_strings = []
file_names = []
iou_thr = 0.6
weights = [1.1, 1.0]

for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        
        if len(predict_list)==0 or len(predict_list)==1:
            continue
            
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
            
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [ ]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('./Ensemble/submission_ensemble.csv')

submission.head()

In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [ ]:
colors = ['cyan', 'lime', 'orange', 'deeppink', 'gold', 'blueviolet', 'red', 'teal', 'hotpink', 'mediumspringgreen']
category = ["General trash", "Paper", "Paper pack", "Metal", "Glass", "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing"]

fig, axes = plt.subplots(1, 3, figsize=(15, 7))

image_id = image_ids[6]
img_path = os.path.join("../dataset/", image_id)
                        
informations = {f"train{idx}" : df for idx, df in enumerate(submission_df)}
informations['ensemble'] = submission
for i, (name, df) in enumerate(informations.items()):
    predict_list = str(df[df['image_id'] == image_id]['PredictionString'].tolist()[0]).split()
    x = i % 3
    # y = i // 3
    img = Image.open(img_path)
    print(predict_list)

    axes[x].imshow(img)
    axes[x].axis('off')
    axes[x].set_title(name)

    for idx in range(len(predict_list)//6):
        label, confidence, x_min, y_min, x_max, y_max = int(predict_list[6*idx + 0]), float(predict_list[6*idx + 1]), float(predict_list[6*idx + 2]), float(predict_list[6*idx + 3]), float(predict_list[6*idx + 4]), float(predict_list[6*idx + 5])
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor=colors[label], facecolor='none')
        axes[x].add_patch(rect)
        axes[x].text(x_min, y_min-10, f"{category[label]}", color=colors[label], fontsize=7, weight='bold')
fig.set_tight_layout
plt.show()